In [ ]:
from statsmodels.tsa.ardl import ARDL
from statsmodels.tools.eval_measures import rmse
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df = pd.read_excel('../dtset.xlsx', sheet_name='filled_data')
df.head()

,bank,quarterly,year,time,NPL,IR,GDP,IIP,INF,CPI,UN,M2,PCR,LTA,ROA,ROE,LDR
0,ABB,1,2012,2012q1,0.0325,0.138700,0.0464,0.0650,0.08,0.1415,0.0222,0.142546,0.0160,0.499451,0.00740,0.065500,0.705935
1,ABB,2,2012,2012q2,0.0325,0.113667,0.0480,0.0802,0.05,0.0690,0.0218,0.142546,0.0153,0.399700,0.00639,0.072606,0.547300
2,ABB,3,2012,2012q3,0.0325,0.092067,0.0505,0.0965,0.04,0.0648,0.0229,0.142546,0.0160,0.499451,0.00740,0.065500,0.705935
3,ABB,4,2012,2012q4,0.0290,0.093667,0.0544,0.0593,0.04,0.0681,0.0212,0.142546,0.0221,0.398600,0.00910,0.083000,0.504700
4,ABB,1,2013,2013q1,0.0325,0.090000,0.0476,0.0516,0.04,0.0664,0.0227,0.142546,0.0221,0.407600,0.00880,0.083000,1.614000


In [3]:
df = df.drop(columns=['CPI', 'ROE'], errors='ignore')

df.head()

,bank,quarterly,year,time,NPL,IR,GDP,IIP,INF,UN,M2,PCR,LTA,ROA,LDR
0,ABB,1,2012,2012q1,0.0325,0.138700,0.0464,0.0650,0.08,0.0222,0.142546,0.0160,0.499451,0.00740,0.705935
1,ABB,2,2012,2012q2,0.0325,0.113667,0.0480,0.0802,0.05,0.0218,0.142546,0.0153,0.399700,0.00639,0.547300
2,ABB,3,2012,2012q3,0.0325,0.092067,0.0505,0.0965,0.04,0.0229,0.142546,0.0160,0.499451,0.00740,0.705935
3,ABB,4,2012,2012q4,0.0290,0.093667,0.0544,0.0593,0.04,0.0212,0.142546,0.0221,0.398600,0.00910,0.504700
4,ABB,1,2013,2013q1,0.0325,0.090000,0.0476,0.0516,0.04,0.0227,0.142546,0.0221,0.407600,0.00880,1.614000


In [6]:
from statsmodels.tsa.ardl import ARDL
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

def run_ardl_forecasting(df, target, exog_vars, lags_dict=None):
    banks = df['bank'].unique()
    ardl_models = {}
    ardl_evals = []

    for bank in banks:
        try:
            bank_df = df[df['bank'] == bank].copy()
            bank_df = bank_df.sort_values("time")

            y = bank_df[target]
            X = bank_df[exog_vars]

            # Check for missing values
            if y.isnull().any() or X.isnull().any().any():
                print(f"ARDL skipped for {bank}: Missing values detected.")
                continue

            # Default lags
            p = 2  # lag for endogenous variable
            q = 1  # lag for exogenous variables

            model = ARDL(endog=y, lags=p, exog=X, exog_lags=q)
            result = model.fit()

            forecast = result.predict(start=len(y) - 4, end=len(y) - 1)
            y_true = y.iloc[-4:]

            mae = mean_absolute_error(y_true, forecast)
            mse = mean_squared_error(y_true, forecast)
            rmse = np.sqrt(mse)
            mape = np.mean(np.abs((y_true - forecast) / y_true.replace(0, np.nan))) * 100

            ardl_models[bank] = result
            ardl_evals.append((bank, mae, mse, rmse, mape))
        except Exception as e:
            print(f"ARDL failed for {bank}: {e}")

    return ardl_models, ardl_evals


In [7]:
exog_vars = ['IR', 'INF', 'GDP', 'IIP', 'UN', 'M2', 'PCR', 'LTA', 'ROA', 'LDR']
target = 'NPL'

ardl_models, ardl_evals = run_ardl_forecasting(df, target, exog_vars)


ARDL failed for ABB: __init__() got an unexpected keyword argument 'exog_lags'
ARDL failed for ACB: __init__() got an unexpected keyword argument 'exog_lags'
ARDL failed for BAB: __init__() got an unexpected keyword argument 'exog_lags'
ARDL failed for BID: __init__() got an unexpected keyword argument 'exog_lags'
ARDL failed for CTG: __init__() got an unexpected keyword argument 'exog_lags'
ARDL failed for EIB: __init__() got an unexpected keyword argument 'exog_lags'
ARDL failed for HDB: __init__() got an unexpected keyword argument 'exog_lags'
ARDL failed for KLB: __init__() got an unexpected keyword argument 'exog_lags'
ARDL failed for MBB: __init__() got an unexpected keyword argument 'exog_lags'
ARDL failed for MSB: __init__() got an unexpected keyword argument 'exog_lags'
ARDL failed for NAB: __init__() got an unexpected keyword argument 'exog_lags'
ARDL failed for OCB: __init__() got an unexpected keyword argument 'exog_lags'
ARDL failed for PGB: __init__() got an unexpected ke